In [1]:
import os 
import time 
import json,math,sys,re
import pandas as pd 

In [3]:
folder = "../"
from dotenv import load_dotenv
env_path = f'{folder}/openai.env'
load_dotenv(dotenv_path=env_path)
OPENAI_API_KEY = os.getenv('SECRET_KEY')
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
inputfname = "Vul_notPriorInCited"
# inputfname = "preE_Vul"
# inputfname = "preE_qp"
fname = f"{inputfname}_GPT4o"
data = pd.read_csv(f"{folder}/QP/{inputfname}.csv")
data = data.rename(columns={'Unnamed: 0':'index'})
data = data.to_dict(orient="records")
data[0]

{'index': 0,
 'verse_id': 'Haggai 2.15 (Geneva) - 1',
 'text': 'Our callings, our labours, our actions, and the workes of our hands are sanctified by it, as Psal. 127.',
 'verse_text': 'and so are all the workes of their hands, and that which they offer here, is vncleane.',
 'label': False}

In [5]:
for idx, item in enumerate(data): 
    item['index'] = idx 
labels = {}

In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
from tqdm import tqdm 
start = 354 # 0 #
end = len(data)
my_qp = data[start:end] 

for qp_pair in tqdm(my_qp):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[ 
            {"role": "user",  
            #  "content": f"You are an English and Latin scholar working on Biblical language in early modern texts. You determine whether the first sentence is a quotation/paraphrase of a given Biblical verse in the SAME LANGUAGE. You give a single TRUE or FALSE output FOR EACH USER MESSAGE without any reasoning. SENTENCE: '{qp_pair['text']}'; 'VERSE: {qp_pair['verse_text']}'"}
            "content":alpaca_prompt.format(
            "You are an English and Latin scholar working on Biblical language in early modern texts. You determine whether the first sentence is a quotation/paraphrase of a given Biblical verse in the SAME LANGUAGE. You give a single TRUE or FALSE output FOR EACH USER MESSAGE without any reasoning.", # instruction
            f"SENTENCE: '{qp_pair['text']}'; 'VERSE: {qp_pair['verse_text']}'", # input
            "", # output - leave this blank for generation!
            )
            }
        ],
        stream=False
    )
    labels[qp_pair['index']] = response.choices[0].message.content

In [10]:
with open(f"{folder}/Early-Modern-Sermons/assets/QP_labels/{fname}.json",'w') as f: 
    json.dump(labels,f) 
len(labels)

355

In [6]:
EEPS_QP_assistant = client.beta.assistants.create(
  name="EEPS Quotations/Paraphrase Identifier",
  instructions="You are an English and Latin scholar working on Biblical language in 16th and 17th century texts from England. You can determine whether a given sentence is a quotation/paraphrase of another given Biblical verse in the SAME LANGUAGE. You give TRUE/FALSE outputs FOR EACH USER MESSAGE without any reasoning.",
  model="gpt-4o",
)

In [7]:
def add_QP(qp_pairs):
    qp = client.beta.threads.create(
        messages=[{"role": "user", 
                   "content": f"SENTENCE: '{qp_pair['text']}'; 'VERSE: {qp_pair['verse_text']}'"} for qp_pair in qp_pairs]
    )
    return qp.id

def run_QP(thread_id):
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=EEPS_QP_assistant.id
    )
    return run.id  

def wait_for_run_completion(thread_id, run_id):
    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run_id
        )
        if run_status.status in ["completed", "failed", "cancelled"]:
            return run_status.status
        time.sleep(1)  

def get_thread_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    return [(msg.role, msg.content[0].text.value) for msg in messages.data]

In [22]:
from tqdm import tqdm 
start = 555 
end = len(data)
my_qp = data[start:end] 

for qp_pair in tqdm(my_qp): 
    thread_id = add_QP([qp_pair]) 
    run_id = run_QP(thread_id)  
    status = wait_for_run_completion(thread_id, run_id) 
    if status == "completed":
        responses = get_thread_messages(thread_id)
        for role, text in responses:
            if role == 'assistant': 
                labels[qp_pair['index']] = text 
    else:
        print(f"Run {qp_pair['index']} failed or was cancelled.")

with open(f"../../DH/Early-Modern-Sermons/assets/QP_labels/{fname}.json",'w') as f: 
    json.dump(labels,f) 
# takes 30 mins for 250 entries; 11 mins for 100 

100%|██████████| 60/60 [07:38<00:00,  7.65s/it]


In [23]:
with open(f"../../DH/Early-Modern-Sermons/assets/QP_labels/{fname}.json",'w') as f: 
    json.dump(labels,f)
len(labels),fname

(615, 'Vul_notPriorInCited_GPT')